<h1>Jack's Car Rentals</h1>

In [1]:
import math 
import numpy as np

In [2]:
maxAtLoc=20
rentFee=10
moveCost=2
maxMove=5
poissonDist={}


In [3]:
def fact(n):
    fact=1
    for i in range(2,n+1):
        fact=fact*i
    return fact

def poisson(lam,n):
    global poissonDist
    key=(n,lam)
    if key not in poissonDist:
        poissonDist[key]=(math.pow(lam,n)*math.exp(-1*lam))/fact(n)
    return poissonDist[key]

In [8]:
#currState=(loc1,loc2)
#action how many cars removed

def valueFunc(currState,action,stateVal):
    ret=0
    
    #moving a car
    loc1=min(currState[0]-action,maxMove)
    loc2=min(currState[1]+action,maxMove)
    ret=ret-moveCost*abs(action)

    for rentL1 in range(0,11):
        for rentL2 in range(0,11):

            probRentL=poisson(rentL1,3)*poisson(rentL2,4)
            possRentalL1=min(rentL1,loc1)
            possRentalL2=min(rentL2,loc2)

            
            retL1=np.random.poisson(3)
            retL2=np.random.poisson(2)

            rew=(possRentalL1+possRentalL2)*rentFee
            finalL1=min(maxAtLoc,loc1-possRentalL1+retL1)
            finalL2=min(maxAtLoc,loc2-possRentalL2+retL2)

            ret=ret+probRentL*(0.9*stateVal[finalL1][finalL2]+rew)

def polIter():
    stateVal=np.zeros((maxAtLoc+1,maxAtLoc+1))
    policy=np.zeros((maxAtLoc+1,maxAtLoc+1))

    steady=False
    while not steady:
        while True:
            pastVal=np.copy(stateVal)
            for x in range(maxAtLoc+1):
                for y in range(maxAtLoc+1):
                    stateVal[x,y]=valueFunc([x][y],policy[x][y],stateVal)
            if np.sum(np.abs(pastVal-stateVal))<0.001:
                break

        steady=True
        for i in range(maxAtLoc+1):
            for j in range(maxAtLoc+1):
                pastAct=policy[i,j]
                actRet=[]
                for action in range(-maxMove,maxMove+1):
                    if (i-action) in range(maxAtLoc+1) and (j+action) in range(maxAtLoc+1):
                        actRet.append(valueFunc([i,j],action,stateVal))
                    else:
                        actRet.append(float(-math.inf))
                newAct=np.argmax(actRet)-maxMove
                policy[i,j]=newAct
                if not newAct==pastAct:
                    steady=False
    return policy,stateVal

In [9]:
policy,stateVal=polIter()
print(f"policy:{policy}, stateVal:{stateVal}")

TypeError: 'int' object is not subscriptable